<a href="https://colab.research.google.com/github/risavh/count_objects/blob/main/Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3206, done.
remote: Counting objects: 100% (3206/3206), done.
remote: Compressing objects: 100% (2711/2711), done.
remote: Total 3206 (delta 854), reused 1371 (delta 451), pack-reused 0
Receiving objects: 100% (3206/3206), 33.40 MiB | 27.94 MiB/s, done.
Resolving deltas: 100% (854/854), done.


In [2]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1683177 sha256=24b708ef4109bfe1c1fe11768f731b15e2388df11b68ecbc1de862173547430d
  Stored in directory: /tmp/pip-ephem-wheel-cache-4wnb9f4u/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22258 sha256=06b345368a96c5efd2ce3fafee81fb00a7c70c58fb1947fbd12f5df10a7f0ec3
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-py3-none-any.whl size=43512 sha256=9071a129d0432366ef62a9e67ceef1cfb6c79a3f40fd98d515fe254833c4ff95
  Stored in directory: /root/.cache/pip/wheels/bc/49/5f/fdb5b9d85055c478213e0158ac122b596816149a02d82e0ab1
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78546 sha256=4fe3087b85f2411ce4eba05940e658ab792e8e209ca612aa2

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [4]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def get_keypoint_tuples(eval_config):
  """Return a tuple list of keypoint edges from the eval config.
  
  Args:
    eval_config: an eval config containing the keypoint edges
  
  Returns:
    a list of edge tuples, each in the format (start, end)
  """
  tuple_list = []
  kp_list = eval_config.keypoint_edge
  for edge in kp_list:
    tuple_list.append((edge.start, edge.end))
  return tuple_list

In [5]:
# @title Choose the model to use, then evaluate the cell.
MODELS = {'centernet_with_keypoints': 'center_net_deepmac_512x512_voc_only_tpu-32', 'centernet_without_keypoints': 'centernet_hg104_512x512_coco17_tpu-8'}

model_display_name = 'centernet_with_keypoints' # @param ['centernet_with_keypoints', 'centernet_without_keypoints']
model_name = MODELS[model_display_name]

In [6]:
# Download the checkpoint and put it into models/research/object_detection/test_data/

if model_display_name == 'centernet_with_keypoints':
  !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz
  !tar -xf centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz
  !mv centernet_hg104_512x512_kpts_coco17_tpu-32/checkpoint models/research/object_detection/test_data/
else:
  !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_coco17_tpu-8.tar.gz
  !tar -xf centernet_hg104_512x512_coco17_tpu-8.tar.gz
  !mv centernet_hg104_512x512_coco17_tpu-8/checkpoint models/research/object_detection/test_data/

--2021-12-15 01:21:25--  http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.218.128, 2607:f8b0:400c:c14::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.218.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453232297 (1.4G) [application/x-tar]
Saving to: ‘centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz’

centernet_hg104_512 100%[===================>]   1.35G  93.3MB/s    in 13s     

2021-12-15 01:21:38 (103 MB/s) - ‘centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz’ saved [1453232297/1453232297]



In [7]:
model_name

'center_net_deepmac_512x512_voc_only_tpu-32'

In [8]:
!ls models/research/object_detection/configs/tf2

center_net_deepmac_1024x1024_coco_tpu-128.config
center_net_deepmac_1024x1024_non_voc_only_tpu-128.config
center_net_deepmac_1024x1024_voc_only_tpu-128.config
center_net_deepmac_512x512_voc_only_tpu-32.config
centernet_hourglass104_1024x1024_coco17_tpu-32.config
centernet_hourglass104_1024x1024_kpts_coco17_tpu-32.config
centernet_hourglass104_512x512_coco17_tpu-8.config
centernet_hourglass104_512x512_kpts_coco17_tpu-32.config
centernet_resnet101_v1_fpn_512x512_coco17_tpu-8.config
centernet_resnet50_v1_fpn_512x512_kpts_coco17_tpu-8.config
centernet_resnet50_v2_512x512_kpts_coco17_tpu-8.config
faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.config
faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.config
faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8.config
faster_rcnn_resnet152_v1_1024x1024_coco17_tpu-8.config
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.config
faster_rcnn_resnet152_v1_800x1333_coco17_gpu-8.config
faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8.config
faster_rcnn_resnet50_v1

In [9]:
pipeline_config = os.path.join('models/research/object_detection/configs/tf2/',
                                model_name + '.config')
model_dir = 'models/research/object_detection/test_data/checkpoint/'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join(model_dir, 'ckpt-0')).expect_partial()

def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [10]:
label_map_path

NameError: ignored

In [12]:
configs

{'eval_config': metrics_set: "coco_detection_metrics"
 metrics_set: "coco_mask_metrics"
 use_moving_averages: false
 include_metrics_per_category: true
 batch_size: 1
 super_categories {
   key: "NonVOC"
   value: "truck,traffic light,fire hydrant,stop sign,parking meter,bench,elephant,bear,zebra,giraffe,backpack,umbrella,handbag,tie,suitcase,frisbee,skis,snowboard,sports ball,kite,baseball bat,baseball glove,skateboard,surfboard,tennis racket,wine glass,cup,fork,knife,spoon,bowl,banana,apple,sandwich,orange,broccoli,carrot,hot dog,pizza,donut,cake,bed,toilet,laptop,mouse,remote,keyboard,cell phone,microwave,oven,toaster,sink,refrigerator,book,clock,vase,scissors,teddy bear,hair drier,toothbrush"
 }
 super_categories {
   key: "VOC"
   value: "person,bicycle,car,motorcycle,airplane,bus,train,boat,bird,cat,dog,horse,sheep,cow,bottle,chair,couch,potted plant,dining table,tv"
 }
 super_categories {
   key: "accessory"
   value: "backpack,umbrella,handbag,tie,suitcase"
 }
 super_categories

In [11]:
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

NotFoundError: ignored